In [ ]:
with open('mesh_rels_dict.dat', 'rb') as f:    
    mesh_rels_dict = pickle.load(f)    # mesh_rels_dict: key: pmid, value: list of combinations of mesh terms for each paper
    f.close()

with open('cits_dict.dat', 'rb') as f2:
    cits_dict = pickle.load(f2)   # cits_dict: key: pmid, value: list of citing pmids for each paper
    f2.close() 
    
with open('refs_dict.dat', 'rb') as f3:
    refs_dict = pickle.load(f3)    # refs_dict: key: pmid, value: list of reference pmids for each paper
    f3.close()

In [ ]:
import numpy as np
from itertools import combinations

m_dict = {}
ED_rels = {}

focal_nodes = list(pmids.keys())
pbar = tqdm(focal_nodes)

for node in pbar:
    cits = cits_dict[node]
    refs = refs_dict[node]
    
    s_rel = mesh_rels_dict[node]
    n_s = len(s_rel)
    
    # ED_s calculation
    sj_rel = set()
    for ref in refs:
        sj_rel.update(mesh_rels_dict[ref])   
    si = s_rel - sj_rel
    sj_old = s_rel & sj_rel
    n_si = len(si)
    n_sj = len(sj_old)
    ED_s = round((n_si - n_sj) / n_s, 5)

    # ED_p calculation
    citing_nodes_fp = [fp_cits for fp_cits in cits]
    N = 0 
    m = 0 
    ED_p = 0
    for c in citing_nodes_fp:
        g_rel = mesh_rels_dict[c]            
        n_g = len(g_rel)
        if n_g > 0:  
            N += 1
            n_gi = len((s_rel & g_rel) - sj_rel)
            n_gj = len(s_rel & g_rel & sj_rel)
            n_gk = len((g_rel & sj_rel) - s_rel)
            n_gn = len(g_rel - sj_rel - s_rel)
            ED_g = round((n_gi + n_gn - n_gj - n_gk) / n_g, 5)
            ED_p += ED_g
        if len(s_rel & g_rel) > 0:
            m += 1
    m_dict[node] = m
       
    if N == 0:
        ED_rel = 0.5 * ED_s + 0.5 * ED_p
        ED_rels[node] = ED_rel
        continue
    
    ED_p = round(ED_p / N, 5)
    ED_rel = 0.5 * ED_s + 0.5 * ED_p
    ED_rels[node] = ED_rel


In [ ]:
import pandas as pd

df_new = pd.DataFrame.from_dict(ED_rel, orient = 'index')
df_new.to_sql('pubmed_ED_noweight', con = engine, if_exists = 'append')

In [ ]:
import numpy as np
import pandas as pd

sql = 'select * from pubmed_ED_noweight'
df_new = pd.read_sql(sql, con = engine)

max_m_years = {}
min_m_years = {}
for y in range(1991,2015,1):
    m_y = np.array(df_new[df_new['pub_year'] == y]['m'].values)
#     m_y = df_new['m'].values
    max_m = np.max(m_y)
    min_m = np.min(m_y)
    max_m_years[y] = max_m
    min_m_years[y] = min_m
# max_m_years = max_m
# min_m_years = min_m

In [ ]:
mED_dict = {}

for i,item in df_new.iterrows():
    pmid = item['pmid']
    ED_rel = item['ED_rel']
    year = item['pub_year']
    m = item['m']
    try: 
        m_weight = round((m - min_m_years[year]) / (max_m_years[year] - min_m_years[year]), 4)
        mED = round(ED_rel * m_weight, 5)
        mED_dict[pmid] = mED
    except:
        continue